# Построение доверительных интервалов

In [36]:
import numpy as np
import scipy

In [12]:
set1 = np.array([1.        , 0.890625  , 0.9375    , 0.796875  , 1.        ,
       0.921875  , 1.        , 0.921875  , 0.59375   , 0.984375  ,
       0.73469388, 0.93877551, 0.87755102, 1.        , 0.95918367,
       0.97959184, 0.85714286, 1.        , 0.97959184, 0.89795918])
set2 = np.array([1.        , 0.890625  , 0.953125  , 0.859375  , 1.        ,
       0.921875  , 1.        , 1.        , 0.90625   , 1.        ,
       0.95918367, 0.97959184, 0.87755102, 1.        , 0.93877551,
       1.        , 0.83673469, 0.95918367, 0.97959184, 0.89795918])

## Оценка среднего

In [18]:
set1 = np.array([1.        , 0.890625  , 0.9375    , 0.796875  , 1.        ,
       0.921875  , 1.        , 0.921875  , 0.59375   , 0.984375  ,
       0.73469388, 0.93877551, 0.87755102, 1.        , 0.95918367,
       0.97959184, 0.85714286, 1.        , 0.97959184, 0.89795918])

Точечная оценка mean и $\sigma^2$

In [13]:
set1_mean = set1.mean() 
set1_std = set1.std(ddof=1) # ddof -- количество степеней свободы. Ставим 1 т.к. 1 степень свободы ушла на оценку среднего
print(set1_mean, set1_std)

0.91356824 0.10454390572446609


$100(1-\alpha)$% доверительный интервал при известной $\sigma$ ($z$-интервал). $$\bar{X}_n \pm z_{1-\frac{\alpha}{2}} \frac{\sigma}{\sqrt{n}}$$

In [14]:
from statsmodels.stats.weightstats import _zconfint_generic
sigma = 0.25
alpha = 0.05
sigma_corr = np.sqrt(sigma/len(set1)) # В силу центральной предельной теоремы

left_bord, right_bord = _zconfint_generic(set1_mean, sigma_corr, alpha, 'two-sided')
print(left_bord, right_bord)

0.6944376048558547 1.1326988751441454
0.91356824
0.11180339887498948
20


$100(1-\alpha)$% доверительный интервал при неизвестной $\sigma$ ($t$-интервал). $$\bar{X}_n \pm t_{1-\frac{\alpha}{2}} \frac{S}{\sqrt{n}}$$

In [17]:
from statsmodels.stats.weightstats import _tconfint_generic
sigma = set1_std
alpha = 0.05
sigma_corr = sigma/np.sqrt(len(set1)) # В силу центральной предельной теоремы

left_bord, right_bord = _tconfint_generic(set1_mean, sigma_corr, len(set1) - 1, alpha, 'two-sided')
print(left_bord, right_bord)

0.8646401860175451 0.962496293982455


## Оценка медианы

В выборке $X^n = (x_1, \dots ,x_n)$ доверительный интервал для медианы: $$x_k < \hat{x} < x_{n - k + 1}$$, где 
$$k = \frac{1}{2}(n - 1.64\sqrt{n} - 1), \text{при} \alpha = 0.1$$
$$k = \frac{1}{2}(n - 1.96\sqrt{n} - 1), \text{при} \alpha = 0.05$$
$$k = \frac{1}{2}(n - 2.58\sqrt{n} - 1), \text{при} \alpha = 0.01$$

На основе [источника](http://www.machinelearning.ru/wiki/index.php?title=%D0%98%D0%BD%D1%82%D0%B5%D1%80%D0%B2%D0%B0%D0%BB%D1%8C%D0%BD%D0%B0%D1%8F_%D0%BE%D1%86%D0%B5%D0%BD%D0%BA%D0%B0)

In [1]:
def calc_t_conf_int(data, alpha=0.05):
    if alpha == 0.01:
        k = 2.58
    elif alpha == 0.05:
        k = 1.96
    elif alpha == 0.1:
        k = 1.64
    else:
        raise ValueError('alpha may be only 0.01, 0.05, 0.1')
    
    n = data.size - 1
    sorted_data = np.sort(data)
    
    index = int((n - k * np.sqrt(n) - 1) / 2)

    left_border = sorted_data[index]
    right_border = sorted_data[n - index + 1]
    
    return (left_border, right_border)

## Оценка доли

In [22]:
np.random.seed(1)
statistical_population = np.random.randint(2, size = 100000) #генеральная совокупность
random_sample = np.random.choice(statistical_population, size = 1000) # выборка из совокупности
#истинное значение доли
statistical_population.mean()

0.49771

Точечная оценка доли

In [23]:
random_sample.mean()

0.502

Доверительный интервал на основе нормального распределения (когда выборка большая и оценка доли близка к 0.5).
$$\hat{p}\pm z_{1-\frac{\alpha}{2}} \sqrt{\frac{\hat{p}\left(1-\hat{p}\right)}{n}}$$

In [26]:
from statsmodels.stats.proportion import proportion_confint
normal_interval = proportion_confint(sum(random_sample), len(random_sample), method = 'normal')
print(normal_interval[0], normal_interval[1])

0.4710104963037765 0.5329895036962234


Доверительный интервал Уилсона (когда выборка мала или оценка доли близка к 0 или 1).
$$\frac1{ 1 + \frac{z^2}{n} } \left( \hat{p} + \frac{z^2}{2n} \pm z \sqrt{ \frac{ \hat{p}\left(1-\hat{p}\right)}{n} + \frac{
z^2}{4n^2} } \right), \;\; z \equiv z_{1-\frac{\alpha}{2}}$$ 

In [27]:
wilson_interval = proportion_confint(sum(random_sample), len(random_sample), method = 'wilson')
print(wilson_interval[0], wilson_interval[1])

0.47106219334840704 0.5329224996176508


Расчёт требуемого размера выборки для доверительного интервала заданной ширины

In [28]:
from statsmodels.stats.proportion import samplesize_confint_proportion
delta = 0.02
n_samples = int(np.ceil(samplesize_confint_proportion(random_sample.mean(), delta / 2)))
print(n_samples)
#проверка
np.random.seed(1)
random_sample = np.random.choice(statistical_population, size = n_samples)
wilson_interval = proportion_confint(sum(random_sample), len(random_sample), method = 'wilson')
print(wilson_interval[1] - wilson_interval[0])

9604
0.019992928893600548


## Доверительные интервалы для двух долей

In [31]:
import pandas as pd

In [29]:
np.random.seed(1)
statistical_population = np.random.randint(2, size = 100000) #генеральная совокупность
random_sample_1 = np.random.choice(statistical_population, size = 1000) # первая выборка из совокупности
random_sample_2 = np.random.choice(statistical_population, size = 1000) # первая выборка из совокупности
#истинное значение доли, а следовательно и доли двух образцов должны совпадать
statistical_population.mean()

0.49771

In [38]:
def make_conf_matrix_ind(sample1, sample2):
    output = pd.DataFrame([[sample1.sum(), len(sample1 - sample1.sum())], 
                           [sample2.sum(), len(sample2 - sample2.sum())]])
    output.columns = ['pos', 'neg']
    output['sum'] = output['pos'] + output['neg']
    return output

def make_conf_matrix_rel(sample1, sample2):
    if len(sample1) != len(sample2):
        raise ValueError('it is not relative samples')
    output = {
        'e': np.logical_and(sample1 > 0.5, sample2 > 0.5).sum(),
        'f': np.logical_and(sample1 > 0.5, sample2 < 0.5).sum(),
        'g': np.logical_and(sample1 < 0.5, sample2 > 0.5).sum(),
        'h': np.logical_and(sample1 < 0.5, sample2 < 0.5).sum(),
        'n': float(len(sample1)),
    }
    
    return output

matrix_ind = make_conf_matrix_ind(random_sample_1, random_sample_2)
matrix_rel = make_conf_matrix_rel(random_sample_1, random_sample_2)

Доверительный интервал для разности долей (независимые выборки)

|  | $X_1$ | $X_2$ | 
|------|------|------|
| 1 | a | b |
| 0 | c | d |
| $\sum$ | $n_1$| $n_2$ |

$$ \hat{p}_1 = \frac{a}{n_1}$$

$$ \hat{p}_2 = \frac{b}{n_2}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\; \hat{p}_1 - \hat{p}_2 \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{\hat{p}_1(1 - \hat{p}_1)}{n_1} + \frac{\hat{p}_2(1 - \hat{p}_2)}{n_2}}$$

In [37]:
def proportions_confint_diff_ind(matrix, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)   
    
    n1 = matrix.iloc[0]['sum']
    n2 = matrix.iloc[1]['sum']
    
    p1 = matrix.iloc[0]['pos'] / n1
    p2 = matrix.iloc[1]['pos'] / n1
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ n1 + p2 * (1 - p2)/ n2)
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ n1 + p2 * (1 - p2)/ n2)
    
    return (left_boundary, right_boundary)

print("confidence interval: [%f, %f]" % proportions_confint_diff_ind(matrix_ind))

confidence interval: [-0.027719, 0.039703]


Доверительный интервал для разности долей (связанные выборки)

  |$X_1$ \ $X_2$ | 1| 0 | $\sum$|
  |--------------|--|---|-------|
  |1  | e | f | e + f|
  |0  | g | h | g + h|
  |$\sum$ | e + g| f + h | n |
  
$$ \hat{p}_1 = \frac{e + f}{n}$$

$$ \hat{p}_2 = \frac{e + g}{n}$$

$$ \hat{p}_1 - \hat{p}_2 = \frac{f - g}{n}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\;  \frac{f - g}{n} \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{f + g}{n^2} - \frac{(f - g)^2}{n^3}}$$

In [39]:
def proportions_confint_diff_rel(matrix, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    n = matrix['n']
    f = matrix['f']
    g = matrix['g']
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

print("confidence interval: [%f, %f]" % proportions_confint_diff_rel(matrix_rel))

confidence interval: [-0.033892, 0.051892]


## Доверительные интервалы на основе бутстрапа

In [41]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

Интервальная оценка среднего

In [46]:
sample_1_median_scores = list(map(np.mean, get_bootstrap_samples(random_sample_1, 1000)))

print("95% confidence interval for the ILEC median repair time:",  stat_intervals(sample_1_median_scores, 0.05))

95% confidence interval for the ILEC median repair time: [0.472    0.532025]


Интервальная оценка разности средних

In [48]:
sample_1_mean_scores = list(map(np.mean, get_bootstrap_samples(random_sample_1, 1000)))
sample_2_mean_scores = list(map(np.mean, get_bootstrap_samples(random_sample_2, 1000)))

delta_mean_scores = list(map(lambda x: x[1] - x[0], zip(sample_1_mean_scores, sample_2_mean_scores)))
print("95% confidence interval for the difference between means",  stat_intervals(delta_mean_scores, 0.05))

95% confidence interval for the difference between medians [-0.053025  0.036025]
